In [1]:
import torch
import numpy as np
from lightning.pytorch import seed_everything
import seisbench.data as sbd

from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from transformers import Wav2Vec2Config
from transformers import Wav2Vec2ForPreTraining as RefWav2Vec2ForPreTraining
from seisLM.model.foundation.multidim_wav2vec2 import MultiDimWav2Vec2ForPreTraining

data = sbd.STEAD()
waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms[0]).unsqueeze(0)


# TODO: Equalize two models at initialization. Currently not the case
# TODO: due to inheritance issues.
# MODEL_NAMES = ["patrickvonplaten/wav2vec2-base-v2", "facebook/wav2vec2-base"]

model_name = "facebook/wav2vec2-base"

# model_output = {}
# for model_name in MODEL_NAMES:
config = Wav2Vec2Config.from_pretrained(model_name)

seed_everything(42)
ref_model = RefWav2Vec2ForPreTraining(config)

seed_everything(42)
model = MultiDimWav2Vec2ForPreTraining(config)

2024-07-11 07:26:22,597 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.
/scicore/home/dokman0000/liu0003/anaconda3/envs/seisbench/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/scicore/home/dokman0000/liu0003/anaconda3/envs/seisbench/lib/python3.9/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Seed set to 42
Seed set to 42


Wav2Vec2GumbelVectorQuantizer(
  (weight_proj): Linear(in_features=512, out_features=640, bias=True)
) is a quantizer


In [2]:
model_param_dict = dict(model.named_parameters())
ref_model_param_dict = dict(ref_model.named_parameters())
  # print(name)

In [3]:
matched_keys = []
unmatched_keys = []
for key in model_param_dict.keys():

  match = torch.equal(model_param_dict[key], ref_model_param_dict[key])
  if match:
    # print(f"matched key: {key}")
    matched_keys.append(key)
  else:
    # print(f"unmatched key: {key}")
    unmatched_keys.append(key)

In [4]:
model_param_dict['quantizer.weight_proj.weight'].std()

tensor(0.9997, grad_fn=<StdBackward0>)

In [5]:
ref_model_param_dict['quantizer.weight_proj.weight'].std()

tensor(0.9997, grad_fn=<StdBackward0>)

In [11]:
matched_keys

['wav2vec2.masked_spec_embed',
 'wav2vec2.feature_extractor.conv_layers.0.conv.weight',
 'wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight',
 'wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias',
 'wav2vec2.feature_extractor.conv_layers.1.conv.weight',
 'wav2vec2.feature_extractor.conv_layers.2.conv.weight',
 'wav2vec2.feature_extractor.conv_layers.3.conv.weight',
 'wav2vec2.feature_extractor.conv_layers.4.conv.weight',
 'wav2vec2.feature_extractor.conv_layers.5.conv.weight',
 'wav2vec2.feature_extractor.conv_layers.6.conv.weight',
 'wav2vec2.feature_projection.layer_norm.weight',
 'wav2vec2.feature_projection.layer_norm.bias',
 'wav2vec2.feature_projection.projection.weight',
 'wav2vec2.feature_projection.projection.bias',
 'wav2vec2.encoder.pos_conv_embed.conv.bias',
 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0',
 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1',
 'wav2vec2.encoder.layer_norm.weight',
 'wav2vec2.enco

In [7]:
from seisLM.model.foundation.quantizer import Wav2Vec2GumbelVectorQuantizer
import transformers.models.wav2vec2.modeling_wav2vec2 as hf_wav2vec2

import ml_collections as mlc

# mlc.hf_wav2vec2.Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base")).quantizer_config

model_name = "facebook/wav2vec2-base"

# model_output = {}
# for model_name in MODEL_NAMES:
config = Wav2Vec2Config.from_pretrained(model_name)



In [8]:
seed_everything(42)
q1 = Wav2Vec2GumbelVectorQuantizer(config)

seed_everything(42)
q2 = hf_wav2vec2.Wav2Vec2GumbelVectorQuantizer(config)

Seed set to 42
Seed set to 42


In [9]:
q1.weight_proj.weight - q2.weight_proj.weight

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<SubBackward0>)